In [29]:
import numpy as np
import pysindy as ps
import networkx as nx
from scipy.integrate import odeint

In [20]:
def bio(data, t):
    F = 5
    B = 1
    R = 1
    
    n = len(data)
    result = np.empty(n)
    
    for i in range(0,n):
        sigma = 0
        for j in range (0,n):
            sigma = sigma + R*(adjacency[i,j])*data[i]*data[j]
        result[i] = F - B*data[i] - sigma
    
    return result

In [44]:
number_of_nodes = 4
adjacency = np.array([
       [0., 0., 0., 1,],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]
])

dt = .02

t_train = np.arange(0, 10, dt)
x0_train = [-8, 8, 27, 5]
x_train = odeint(bio, x0_train, t_train)


library_functions = [
    lambda : 1,
    lambda x : x,
    lambda x,y : x*y
]
library_function_names = [
    lambda : 1,
    lambda x : x,
    lambda x,y : '' + x + '.' + y
]
custom_library = ps.CustomLibrary(
    library_functions=library_functions, function_names=library_function_names
)

model = ps.SINDy(feature_library=custom_library)
model.fit(x_train, t=dt)
model.print()

x0' = 5.379 1 + -0.588 x1 + 0.861 x2 + -1.260 x3 + -0.003 x0.x1 + 0.006 x0.x2 + -0.997 x0.x3 + 0.010 x1.x2 + -0.142 x1.x3
x1' = 4.448 1 + 0.517 x0 + -8.801 x1 + -3.181 x2 + 4.362 x3 + 2.741 x0.x1 + -0.522 x0.x3 + 0.375 x1.x2
x2' = -9.860 1 + 7.559 x0 + -15.222 x1 + -8.985 x2 + 11.826 x3 + 2.700 x0.x1 + -1.624 x0.x3 + 0.392 x1.x2 + 4.304 x1.x3
x3' = 4.238 1 + -0.152 x0 + -0.655 x2 + -0.013 x0.x2 + -1.045 x0.x3 + 0.188 x1.x3


In [189]:
def find_steady_state(X):
    prev_state = X[0]
    for i in range(1,len(X)):
        state = X[i]
        max_diff = np.amax(np.subtract(state, prev_state))
        prev_state = state
        if max_diff < 0.0001:
            return state

real_steady_state = find_steady_state(x_train)        
        
def bio_sindy(data, t):
    # custom_library.get_feature_names()
    cf = [1, data[0], data[1], data[2], data[3], 
     data[0]*data[1], data[0]*data[2], data[0]*data[3],
     data[1]*data[2], data[1]*data[3], data[2]*data[3]]
    
    result = np.empty(len(data))
    for index in range(0, len(data)):
        sum = 0
        for i in range(0, len(cf)):
            sum += coefficients[index][i] * cf[i]
        result[index] = sum
    return result

coefficients = model.coefficients()
x_test = odeint(bio_sindy, x0_train, t_train)
sindy_steady_state = find_steady_state(x_test)

print('real_steady_state:', real_steady_state)
print('sindy_steady_state:', sindy_steady_state)

real_steady_state: [1.78790659 1.786349   1.79624565 1.79467798]
sindy_steady_state: [1.76711039 1.78264138 1.81015107 1.81240566]


In [190]:
perturbation = np.array([1., 0., 0., 0.])
perturbed = np.add(sindy_steady_state, perturbation)

t_perturbed = np.arange(0, 10, dt)
x_perturbed = odeint(bio_sindy, perturbed, t_perturbed)
perturbed_steady_state = find_steady_state(x_perturbed)

print('perturbed_steady_state:', perturbed_steady_state)

perturbed_steady_state: [1.77997345 1.79230681 1.80074295 1.80110508]


In [194]:
def flow(X):
    prev_state = X[0]
    for i in range(1,len(X)):
        state = X[i]
        diff = np.subtract(state, prev_state)
        prev_state = state
        print(diff)
        if np.amax(diff) < 0.001:
            break

flow(x_perturbed)

[-0.03287346  0.08150913  0.16552055 -0.04051209]
[-0.02709948  0.06836013  0.12450236 -0.03880484]
[-0.02215827  0.05771405  0.09053847 -0.03702363]
[-0.01794111  0.04917769  0.06260748 -0.03519632]
[-0.01435294  0.04241176  0.03981284 -0.03334697]
[-0.01131095  0.03712446  0.02137106 -0.03149607]
[-0.00874311  0.03306497  0.00659909 -0.02966081]
[-0.00658707  0.03001906 -0.00509466 -0.02785535]
[-0.0047888   0.02780378 -0.01422134 -0.02609109]
[-0.00330166  0.02626377 -0.02121995 -0.02437696]
[-0.00208539  0.02526744 -0.02646596 -0.02271965]
[-0.00110527  0.0247038  -0.03027886 -0.02112395]
[-0.00033138  0.02447952 -0.03292919 -0.01959295]
[ 0.00026207  0.02451642 -0.03464489 -0.01812832]
[ 0.00069731  0.02474926 -0.035617   -0.01673051]
[ 0.00099351  0.02512369 -0.03600478 -0.01539902]
[ 0.00116724  0.02559462 -0.03594031 -0.0141325 ]
[ 0.00123281  0.02612471 -0.0355324  -0.01292901]
[ 0.00120264  0.02668308 -0.03487017 -0.0117861 ]
[ 0.00108749  0.02724426 -0.03402607 -0.01070096]
